In [10]:
import pymongo
from pymongo import MongoClient, UpdateOne
import json
from datetime import datetime
from collections import OrderedDict

In [11]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [12]:
# assign the database to a variable name
db = mongo.chicago_bikes

In [13]:
#define collection variables
divvy_rides = db['divvy_ridedata']
weather_daily = db['weather_daily']
divvy_ridedata_merged = db['divvy_ridedata_merged']

In [31]:
divvy_ridedata_merged = db["divvy_ridedata_merged"]

# Define the aggregation pipeline
pipeline = [
    {
        "$group": {
            "_id": {
                "year": {"$year": {"$toDate": "$started_at"}},
                "month": {"$month": {"$toDate": "$started_at"}}
            },
            "total_rides": {"$sum": 1}
        }
    },
    {
        "$project": {
            "_id": 0,
            "year": "$_id.year",
            "month": "$_id.month",
            "total_rides": 1
        }
    },
    {
        "$sort": {"year": 1, "month": 1}
    }
]

# Execute the aggregation pipeline and write to a new collection
divvy_rides_by_month = db["divvy_rides_by_month"]
aggregated_result = divvy_ridedata_merged.aggregate(pipeline, allowDiskUse=True, collation=None)

for doc in aggregated_result:
    divvy_rides_by_month.insert_one(doc)

print("Aggregation result has been written to the new collection.")

Aggregation result has been written to the new collection.


In [32]:
print(db.list_collection_names())

['divvy_rides_by_month', 'weather_daily', 'divvy_ridedata_merged', 'divvy_ridedata']


In [33]:
divvy_rides_by_month.find_one()

{'_id': ObjectId('64e7f84cf51c9daca89e434c'),
 'total_rides': 103770,
 'year': 2022,
 'month': 1}